In [1]:
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import statsmodels.api as sm

In [2]:
from TDSCoinbaseData import TDSCoinbaseData
from TDSTickGenerator import TDSTickGenerator
from TDSTransactionTracker import TDSTransactionTracker
import logging
logging.getLogger().setLevel(level=logging.ERROR)

import warnings
warnings.filterwarnings('ignore')

In [3]:
# instantiate our data interface
cb_obj = TDSCoinbaseData(cache_path='../data', notebook_logging=True)
# Set the start and end date
start_date = '20201231'
end_date = '20201231'

# List all of the products to be used in the strategy. Be sure to list all products you may use, these can't be updated later
products = ['BTC-USD', 'ETH-USD', 'BTC-EUR', 'ETH-EUR', 'LTC-USD', 'LTC-EUR']

In [4]:
MAX_TRAINNING_SIZE = 360
SIZE_LIMIT_REACHED = False

In [5]:
def append_ticks(tick, usd_btc, eth_btc, ltc_btc, 
                 max_r=SIZE_LIMIT_REACHED, scale_btc=1000000, scale_eth=10000, scale_ltc=10000):
    x = tick.p.btc_usd
    y = tick.p.eth_usd
    z = tick.p.ltc_usd
    usd_btc = usd_btc.append({'open': scale_btc/x.open, 'close': scale_btc/x.close, 
                      'volume': x.volume, 'datetime': tick.datetime}, ignore_index=True)
    eth_btc = eth_btc.append({'open': scale_eth * y.open/x.open, 'close': scale_eth *  y.close/x.close, 
                      'volume': y.volume, 'datetime': tick.datetime}, ignore_index=True)
    ltc_btc = ltc_btc.append({'open': scale_ltc * z.open/x.open, 'close': scale_ltc * z.close/x.close, 
                      'volume': z.volume, 'datetime': tick.datetime}, ignore_index=True)
    # Checks if size limit has been reached
    if (SIZE_LIMIT_REACHED or len(usd_btc.index) > MAX_TRAINNING_SIZE) :
        usd_btc.iloc[1:]
        eth_btc.iloc[1:]
        ltc_btc.iloc[1:]
        max_r = True
        
    return(usd_btc, eth_btc, ltc_btc, max_r)

In [6]:
def calculate_time(pddatetime, numtime, plus=True):
    if plus:
        return pddatetime.to_pydatetime() + timedelta(minutes=numtime)
    else: 
        return pddatetime.to_pydatetime() - timedelta(minutes=numtime)

# frame -- dataframe
# minute -- the number of minutes to be predicted
def predict(frame, minute=3) :
    frame = frame.set_index('datetime')
    y = frame['open'].resample('1T').mean()
    l = frame['close'].resample('1T').mean()
    v = frame['volume'].resample('1T').mean()
    merged = pd.merge(y, l, on='datetime', how="outer")
    frame = pd.merge(merged, v, on='datetime', how="outer")

    mod = sm.tsa.VARMAX(frame[['open', 'close', 'volume']], order=(1, 0), error_cov_type='diagonal')
    results = mod.fit(maxiter=50, disp=False)
    #print(results.summary().tables[1])
    newtime = calculate_time(tick.datetime, minute)
    pred = results.get_prediction(start=pd.to_datetime(frame.index[-1]), end=newtime, dynamic=False)
    pred_ci = pred.conf_int()
    #print(pred.predicted_mean)
#     print(frame.iloc[-1])
    return (pred)
# Call this function for btc prediction
#     minute -- the number of minutes to be predicted
# def predict_btc(minute) :
#     return predict(usd_btc, minute)
    
# def predict_eth(minute) :
#     return predict(eth_btc, minute)

In [7]:
start_date = '20201201'
end_date = '20201231'

# Instantiate our transaction tracker. As required, we start with 1.0 BTC
trans_tracker = TDSTransactionTracker(start_date, end_date, holdings={'BTC' : 1.0})

# Instantiate a tick generator
tick_gen = TDSTickGenerator(cb_obj, products, start_date, end_date, interval=60)
tick = tick_gen.get_tick()

# Dataframe for history
d = {'open': [], 'close': [], 'volume': [], 'datetime': []}
usd_btc = pd.DataFrame(data=d)
eth_btc = pd.DataFrame(data=d)
ltc_btc = pd.DataFrame(data=d)

#loads initial value for testing
for i in range(0, 150):
    tick = tick_gen.get_tick()
    (usd_btc, eth_btc, ltc_btc, SIZE_LIMIT_REACHED) = append_ticks(tick, usd_btc, eth_btc, ltc_btc)

In [8]:
def get_coin_ratio(tick, holdings):
    btc = holdings.get("BTC") if "BTC" in holdings else 0
    eth = tick.p.eth_usd.close/tick.p.btc_usd.close * holdings.get("ETH") if "ETH" in holdings else 0
    ltc = tick.p.ltc_usd.close/tick.p.btc_usd.close * holdings.get("LTC") if "LTC" in holdings else 0
    usd = holdings.get("USD")/tick.p.btc_usd.close if "USD" in holdings else 0
    res = np.array([usd, eth, ltc, btc])
    return sum(res), res/sum(res)

In [9]:
def trade_ratio(curr_port, ideal_port, tick, trans_tracker, curr_hold_in_BTC):
    gap = (ideal_port - curr_port)[1:]
    gap_in_usd = gap * curr_hold_in_BTC * tick.p.btc_usd.close
    names = np.array(['ETH-USD', 'LTC-USD', 'BTC-USD'])
    names = names[np.argsort(gap_in_usd)]
    gap_in_usd = np.sort(gap_in_usd)
    # holdings = trans_tracker.get_holdings()
    
    for idx in range(len(names)):
        product = names[idx]
        change = gap_in_usd[idx]
        attr = getattr(tick.p, product.lower().replace('-', '_'))
        # curr_hold = trans_tracker.get_holdings().get(product) if product in 
        if change < 0:
            valid_vol = min(- change / attr.close, attr.volume * 0.5)
#             print(product[:3], trans_tracker.get_holdings().get(product[:3]))
            valid_vol = min(valid_vol, trans_tracker.get_holdings().get(product[:3])) // 0.000001 / 1000000
#             print(- change / attr.close, attr.volume * 0.5)
            trans_tracker.make_trade(tick, product, 'sell', valid_vol)
        elif change > 0:
            valid_amt = min(change, trans_tracker.get_holdings().get("USD"))
#             print(attr.volume)
            valid_amt = min(valid_amt, attr.volume * attr.close * 0.5) // 0.000001 / 1000000
#             print(valid_amt)
            trans_tracker.make_trade(tick, product, 'buy', valid_amt)
            
    # print(names, gap_in_usd)
    
#     btc_gap = (usd_val * b - btc) / tick.p.btc_usd.close
#     eth_gap = (usd_val * e - eth) / tick.p.eth_usd.close
#     ltc_gap = (usd_val * l - ltc) / tick.p.ltc_usd.close
    
#     if (btc_gap > 0):
#         trans_tracker.make_trade(tick, 'BTC-USD', 'sell', btc_gap / tick.p.btc_usd.close)
#     if (eth_gap > 0):
#         trans_tracker.make_trade(tick, 'ETH-USD', 'sell', eth_gap / tick.p.eth_usd.close)
#     if (ltc_gap > 0):
#         trans_tracker.make_trade(tick, 'LTC-USD', 'sell', ltc_gap / tick.p.ltc_usd.close)
        
#     if (btc_gap < 0):
#         trans_tracker.make_trade(tick, 'BTC-USD', 'buy', min(btc_gap, trans_tracker.get_holdings().get("USD")))
#     if (eth_gap < 0):
#         trans_tracker.make_trade(tick, 'ETH-USD', 'buy', min(eth_gap, trans_tracker.get_holdings().get("USD")))
#     if (ltc_gap < 0):
#         trans_tracker.make_trade(tick, 'LTC-USD', 'buy', min(ltc_gap, trans_tracker.get_holdings().get("USD")))

In [10]:
def trade_eth(tick, bu, eu, be, ee, has_usd):
        # Amount of ETH and BTC to trade at most due to volume restriction
        num_eth = min(tick.p.eth_usd.volume, tick.p.eth_eur.volume) / 2
        num_btc = num_eth * ee / be
        if (num_btc > min(tick.p.btc_usd.volume, tick.p.btc_eur.volume) / 2):
            num_btc = min(num_eth * ee / be, min(tick.p.btc_usd.volume, tick.p.btc_eur.volume) / 2)
            num_eth = num_btc * be / ee

        # Buy Bitcoin with USD
        trans_tracker.make_trade(tick, 'BTC-USD', 'buy', min(num_btc * bu, has_usd))
        # print(trans_tracker.get_holdings())
        # Sell Bitcoin for EUR
        trans_tracker.make_trade(tick, 'BTC-EUR', 'sell', min(num_btc, trans_tracker.get_holdings().get("BTC")))
        # print(trans_tracker.get_holdings())
        # Buy ETH with EUR
        trans_tracker.make_trade(tick, 'ETH-EUR', 'buy', min(num_eth * ee, trans_tracker.get_holdings().get("EUR")))
        # print(trans_tracker.get_holdings())
        # Sell ETH with USD
        trans_tracker.make_trade(tick, 'ETH-USD', 'sell', trans_tracker.get_holdings().get("ETH"))
        # print(trans_tracker.get_holdings())
        # Buy Bitcoin with USD
        trans_tracker.make_trade(tick, 'BTC-USD', 'buy', min(num_btc * bu, trans_tracker.get_holdings().get("USD")))
        # print(trans_tracker.get_holdings())

In [11]:
def trade_ltc(tick, bu, lu, be, le, has_usd):
        # Amount of LTC and BTC to trade at most due to volume restriction
        num_ltc = min(tick.p.ltc_usd.volume, tick.p.ltc_eur.volume) / 2
        num_btc = num_ltc * le / be
        if (num_btc > min(tick.p.btc_usd.volume, tick.p.btc_eur.volume) / 2):
            num_btc = min(num_ltc * le / be, min(tick.p.btc_usd.volume, tick.p.btc_eur.volume) / 2)
            num_ltc = num_btc * be / le

        # Buy Bitcoin with USD
        trans_tracker.make_trade(tick, 'BTC-USD', 'buy', min(num_btc * bu, has_usd))
        # print(trans_tracker.get_holdings())
        # Sell Bitcoin for EUR
        trans_tracker.make_trade(tick, 'BTC-EUR', 'sell', min(num_btc, trans_tracker.get_holdings().get("BTC")))
        # print(trans_tracker.get_holdings())
        # Buy LTC with EUR
        trans_tracker.make_trade(tick, 'LTC-EUR', 'buy', min(num_ltc * le, trans_tracker.get_holdings().get("EUR")))
        # print(trans_tracker.get_holdings())
        # Sell LTC with USD
        trans_tracker.make_trade(tick, 'LTC-USD', 'sell', trans_tracker.get_holdings().get("LTC"))
        # print(trans_tracker.get_holdings())
        # Buy Bitcoin with USD
        trans_tracker.make_trade(tick, 'BTC-USD', 'buy', min(num_btc * bu, trans_tracker.get_holdings().get("USD")))
        # print(trans_tracker.get_holdings())

In [12]:
def trades(tick, trans_tracker, usd_btc, eth_btc, ltc_btc, SIZE_LIMIT_REACHED, scale_btc=1000000, scale_eth=10000, scale_ltc=10000):
    
    # Get current prices 
    x_price = scale_btc / tick.p.btc_usd.close
    y_price = scale_eth * tick.p.eth_usd.close / tick.p.btc_usd.close
    z_price = scale_ltc * tick.p.ltc_usd.close / tick.p.btc_usd.close

    curr_hold_in_BTC, curr_port = get_coin_ratio(tick, trans_tracker.get_holdings())
    # print(count, curr_hold_in_BTC)
    
    # Get predicted values
    pred_usd = predict(usd_btc, 1)
    pred_eth = predict(eth_btc, 1)
    pred_ltc = predict(ltc_btc, 1)
    
    # Find which one increases the most
    changes = np.array([(pred_usd.predicted_mean["close"][-1]-x_price)/x_price, 
               (pred_eth.predicted_mean["close"][-1]-y_price)/y_price, 
               (pred_ltc.predicted_mean["close"][-1]-z_price)/z_price, 
                       0])

    
    # Obtain ideal portfolio
    if max(changes) - sum(curr_port * changes) > 0.005:
        ideal_port = np.zeros_like(changes)
        ideal_port[changes.argmax(0)] = 1

        # print(count, curr_port, ideal_port)
        trade_ratio(curr_port, ideal_port, tick, trans_tracker, curr_hold_in_BTC)
    
    (usd_btc, eth_btc, ltc_btc, SIZE_LIMIT_REACHED) = append_ticks(tick, usd_btc, eth_btc, ltc_btc)

In [13]:
hr=0
count = 0
benchmark = 1.4
while tick is not None:
    curr_hold_in_BTC, curr_port = get_coin_ratio(tick, trans_tracker.get_holdings())
    count += 1
    print(count, curr_hold_in_BTC)
    if (tick and not hr == 0 and hr != tick.datetime.to_pydatetime().hour):
        print("Time:", tick.datetime.to_pydatetime().hour)
        print("holdings", trans_tracker.get_holdings())
        hr = tick.datetime.to_pydatetime().hour
        # 10% chance of attempting to trade
#         no trade for this tick
    if (not tick.p.btc_usd.volume and not tick.p.eth_usd.volume and not tick.p.ltc_usd.volume):
        continue

    bu = tick.p.btc_usd.close
    eu = tick.p.eth_usd.close
    lu = tick.p.ltc_usd.close
    be = tick.p.btc_eur.close
    ee = tick.p.eth_eur.close
    le = tick.p.ltc_eur.close

    has_usd = trans_tracker.get_holdings().get("USD")

    # Check exchange ratios for BTC - ETH and BTC - LTC
    be_ratio = (be * eu) / (ee * bu) * 100 - 100
    bl_ratio = (be * lu) / (le * bu) * 100 - 100

    # Check if it is an arbitrage opportunity for BTC - ETH or BTC - LTC
    if (be_ratio < benchmark and bl_ratio < benchmark):
        trades(tick, trans_tracker, usd_btc, eth_btc, ltc_btc, SIZE_LIMIT_REACHED)
        tick = tick_gen.get_tick()
        continue

    # Check which would guarantee larger profit
    if (be_ratio > bl_ratio):
        trade_eth(tick, bu, eu, be, ee, has_usd)
    else:
        trade_ltc(tick, bu, lu, be, le, has_usd)

    tick = tick_gen.get_tick()

1 1.0
2 1.0
3 1.0
4 1.0
5 1.0
6 1.0
7 1.0
8 1.0
9 1.0
10 1.0
11 1.0
12 1.0
13 1.0
14 1.0
15 1.0
16 1.0
17 1.0
18 1.0
19 1.0
20 1.0
21 1.0
22 1.0
23 1.0
24 1.0
25 1.0
26 0.9984889391184057
27 0.9973468324136999
28 0.9981290580757431
29 0.9993140959067514
30 0.998519870581914
31 0.9992770208829912
32 0.9993349200718062
33 1.0011591265680921
34 1.0006348447805553
35 1.0008834152891972
36 1.001415098075866
37 1.0026509029790276
38 1.0022190461675538
39 1.0016967158920298
40 1.0017538750690989
41 1.001912628066696
42 1.003381049938201
43 1.0028385827703374
44 1.0027787426195671
45 1.0024060532616053
46 1.0031665517517594
47 1.0031445440000326
48 1.0036637747252466
49 1.0033764416146023
50 1.0024980495669216
51 1.0022440785908895
52 1.003976391478845
53 1.0034194542811024
54 1.0039779294195261
55 1.004097903311386
56 1.004448247560664
57 1.0046145285860852
58 1.0034214076770132
59 1.0040726555263086
60 1.0037467190072167
61 1.0039030709619
62 1.0050447582066726
63 1.0051337981091701
64 1.005

KeyboardInterrupt: 

In [ ]:
arbitrage(1.4, tick)

In [ ]:
trans_tracker.get_holdings()

In [ ]:
get_coin_ratio(tick, trans_tracker.get_holdings())